In [1]:
import sys
sys.path.append("ALL_IN_ONE/Pythia8")
sys.path.append("ALL_IN_ONE/LSD")
sys.path.append("ALL_IN_ONE/FIT_FUNC")
import os
from tqdm import tqdm
import ALL_IN_ONE.Pythia8.loop as loop
import ALL_IN_ONE.Pythia8.one_key_run as okr 
import ALL_IN_ONE.Pythia8.run_save as rs
import ALL_IN_ONE.Pythia8.analyse_data as ad
import ALL_IN_ONE.Pythia8.detection as detection
import ALL_IN_ONE.Pythia8.functions_for_calculation as fcal
import ALL_IN_ONE.Pythia8.plotting as fplot
# from detection import detect_percise_and_combine, detect_r_and_combine
# from analyse_data import see_ctau_2, print_max_min_log10_sin_theta_2
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt
from tqdm import tqdm
# import tkinter as tk
# from tkinter import ttk
from threading import Thread
from tqdm import tqdm
import numpy as np
from PIL import Image
# from run_save import run_save_main41_txt, runtxt_to_csv, add_typed_in_data, add_whether_in_the_detector, add_whether_in_the_detector_without_angle
import os
from ALL_IN_ONE.Pythia8.plotting import plot_llp_decay_in_the_detector
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import subprocess
from datetime import datetime
import os
import subprocess
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import pandas as pd
import random
from scipy.interpolate import interp1d
from tqdm import tqdm
import glob
from scipy.interpolate import PchipInterpolator
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
# from functions_for_run import mkdir_1
import re
# from functions_for_read import get_llp
# from functions_for_calculation import calculate_decay_position, whether_in_the_detector_by_position, whether_in_the_detector_by_r


In [2]:
hep.style.use(hep.style.ALICE)

In [18]:
fcal.f(1.28**2/4.18**2)

np.float64(0.5060976140045761)

In [9]:
def plot_with_envelope_tanb_MATHU_CODEXb_H(csv_file, interpolation='cubic', output_file='envelope_plot.png', visible_br_lowest=0.76):
    # 删除离群点
    df = pd.read_csv(csv_file)


    # df['significance_with_4pi'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3
    # df['significance_without_4pi'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * df['visible_br_without_4pi']
    df['significance_lowest_CODEX-b'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * visible_br_lowest
    df['significance_lowest_MATHUSLA'] = df['detector_acceptance_MATHUSLA']* fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * visible_br_lowest
    # df['simple_br'] = df['detector_acceptance'] * fcal.calculate_Br(df['m'], df['tanb'], 0.104, 0.653) * 300 * df['Cross_section_fb'] * 1e3 * df['visible_br_without_4pi']
    # fig, axs = plt.subplots(2, 2, figsize=(20, 15))
    # df_threshold = df[df['significance_with_4pi'] > 3]
    # df_threshold_without_4pi = df[df['significance_without_4pi'] > 3]
    df_threshold_lowest_CODEX_b = df[df['significance_lowest_CODEX-b'] > 3]
    df_threshold_lowest_MATHUSLA = df[df['significance_lowest_MATHUSLA'] > 3]
    # df_threshold_simple_br = df[df['simple_br'] > 3]
    def add_envelope(ax, x, y, label, color, interpolation='cubic'):
        """
        Add an envelope to the given axis and interpolate the upper and lower envelopes.

        Parameters:
        - ax: matplotlib axis object
        - x: x data (m)
        - y: y data (sin^2(tanb))
        - label: label for the envelope
        - color: color of the envelope
        - interpolation: interpolation method ('cubic' or 'quadratic')
        """
        # Sort data by x
        sorted_indices = np.argsort(x)
        x = np.array(x)[sorted_indices]
        y = np.array(y)[sorted_indices]

        # Sort upper and lower envelope points
        unique_x = np.unique(x)
        upper_envelope_x = []
        upper_envelope_y = []
        lower_envelope_x = []
        lower_envelope_y = []

        for ux in unique_x:
            mask = x == ux
            upper_envelope_x.append(ux)
            upper_envelope_y.append(y[mask].max())  
            lower_envelope_x.append(ux)
            lower_envelope_y.append(y[mask].min())  

        # Interpolate the upper and lower envelope
        x_interp = np.linspace(min(upper_envelope_x), max(upper_envelope_x), 500)
        upper_interp = interp1d(upper_envelope_x, upper_envelope_y, kind=interpolation, fill_value="extrapolate")
        lower_interp = interp1d(lower_envelope_x, lower_envelope_y, kind=interpolation, fill_value="extrapolate")

        # generate interpolated values
        y_upper = upper_interp(x_interp)
        y_lower = lower_interp(x_interp)

        ax.plot(x_interp, y_upper, color=color, linestyle='--', alpha=0.8)
        ax.plot(x_interp, y_lower, color=color, linestyle='--', alpha=0.8)
        ax.fill_between(x_interp, y_lower, y_upper, color=color, alpha=0.1, label=f'Envelope Area ({label})')

    # Remove rows with NaN or infinite values in relevant columns
    # df_threshold_lowest_br = df_threshold_lowest_br.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance', 'significance_lowest_br'])
    df_threshold_lowest_CODEX_b = df_threshold_lowest_CODEX_b.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance', 'significance_lowest_CODEX-b'])
    df_threshold_lowest_MATHUSLA = df_threshold_lowest_MATHUSLA.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance_MATHUSLA', 'significance_lowest_MATHUSLA'])
    # df_threshold_without_4pi = df_threshold_without_4pi.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance', 'significance_without_4pi'])
    # Subplot 1: No 4Pi
    plt.figure(figsize=(15, 15))
    plt.scatter(df_threshold_lowest_CODEX_b['m'], df_threshold_lowest_CODEX_b['tanb'], label='CODEX-b', color='green', s=20, alpha=0.3)
    add_envelope(plt, df_threshold_lowest_CODEX_b['m'], df_threshold_lowest_CODEX_b['tanb'], 'CODEX-b', 'green', interpolation)
    plt.scatter(df_threshold_lowest_MATHUSLA['m'], df_threshold_lowest_MATHUSLA['tanb'], label='MATHUSLA', color='blue', s=20, alpha=0.3)
    add_envelope(plt, df_threshold_lowest_MATHUSLA['m'], df_threshold_lowest_MATHUSLA['tanb'], 'MATHUSLA', 'blue', interpolation)
    plt.xlim(min(df['m']), 10)
    # axs[0, 0].set_ylim(1e-14, 1e-6)
    # plt.title(r'A, mHC = 90GeV, cos($\alpha$-$\beta$) = 0', fontsize=20)
    plt.title(r'H, mHC = 600GeV, cos($\alpha$-$\beta$) = 1/tan$\beta$', fontsize=20)
    plt.xlabel('mass / GeV', fontsize=15)
    plt.ylabel(r'tan$\beta$', fontsize=15)
    plt.yscale('log')
    plt.xscale('log')
    plt.legend()
    plt.tight_layout()
    plt.savefig(output_file)
    plt.show()
    plt.close()
    
def plot_with_envelope_tanb_MATHU_CODEXb_A(csv_file, interpolation='cubic', output_file='envelope_plot.png', visible_br_lowest=0.76):
    # 删除离群点
    df = pd.read_csv(csv_file)


    # df['significance_with_4pi'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3
    # df['significance_without_4pi'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * df['visible_br_without_4pi']
    df['significance_lowest_CODEX-b'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * visible_br_lowest
    df['significance_lowest_MATHUSLA'] = df['detector_acceptance_MATHUSLA']* fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * visible_br_lowest
    # df['simple_br'] = df['detector_acceptance'] * fcal.calculate_Br(df['m'], df['tanb'], 0.104, 0.653) * 300 * df['Cross_section_fb'] * 1e3 * df['visible_br_without_4pi']
    # fig, axs = plt.subplots(2, 2, figsize=(20, 15))
    # df_threshold = df[df['significance_with_4pi'] > 3]
    # df_threshold_without_4pi = df[df['significance_without_4pi'] > 3]
    df_threshold_lowest_CODEX_b = df[df['significance_lowest_CODEX-b'] > 3]
    df_threshold_lowest_MATHUSLA = df[df['significance_lowest_MATHUSLA'] > 3]
    # df_threshold_simple_br = df[df['simple_br'] > 3]
    def add_envelope(ax, x, y, label, color, interpolation='cubic'):
        """
        Add an envelope to the given axis and interpolate the upper and lower envelopes.

        Parameters:
        - ax: matplotlib axis object
        - x: x data (m)
        - y: y data (sin^2(tanb))
        - label: label for the envelope
        - color: color of the envelope
        - interpolation: interpolation method ('cubic' or 'quadratic')
        """
        # Sort data by x
        sorted_indices = np.argsort(x)
        x = np.array(x)[sorted_indices]
        y = np.array(y)[sorted_indices]

        # Sort upper and lower envelope points
        unique_x = np.unique(x)
        upper_envelope_x = []
        upper_envelope_y = []
        lower_envelope_x = []
        lower_envelope_y = []

        for ux in unique_x:
            mask = x == ux
            upper_envelope_x.append(ux)
            upper_envelope_y.append(y[mask].max())  
            lower_envelope_x.append(ux)
            lower_envelope_y.append(y[mask].min())  

        # Interpolate the upper and lower envelope
        x_interp = np.linspace(min(upper_envelope_x), max(upper_envelope_x), 500)
        upper_interp = interp1d(upper_envelope_x, upper_envelope_y, kind=interpolation, fill_value="extrapolate")
        lower_interp = interp1d(lower_envelope_x, lower_envelope_y, kind=interpolation, fill_value="extrapolate")

        # generate interpolated values
        y_upper = upper_interp(x_interp)
        y_lower = lower_interp(x_interp)

        ax.plot(x_interp, y_upper, color=color, linestyle='--', alpha=0.8)
        ax.plot(x_interp, y_lower, color=color, linestyle='--', alpha=0.8)
        ax.fill_between(x_interp, y_lower, y_upper, color=color, alpha=0.1, label=f'Envelope Area ({label})')

    # Remove rows with NaN or infinite values in relevant columns
    # df_threshold_lowest_br = df_threshold_lowest_br.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance', 'significance_lowest_br'])
    df_threshold_lowest_CODEX_b = df_threshold_lowest_CODEX_b.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance', 'significance_lowest_CODEX-b'])
    df_threshold_lowest_MATHUSLA = df_threshold_lowest_MATHUSLA.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance_MATHUSLA', 'significance_lowest_MATHUSLA'])
    # df_threshold_without_4pi = df_threshold_without_4pi.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance', 'significance_without_4pi'])
    # Subplot 1: No 4Pi
    plt.figure(figsize=(15, 15))
    plt.scatter(df_threshold_lowest_CODEX_b['m'], df_threshold_lowest_CODEX_b['tanb'], label='CODEX-b', color='green', s=20, alpha=0.3)
    add_envelope(plt, df_threshold_lowest_CODEX_b['m'], df_threshold_lowest_CODEX_b['tanb'], 'CODEX-b', 'green', interpolation)
    plt.scatter(df_threshold_lowest_MATHUSLA['m'], df_threshold_lowest_MATHUSLA['tanb'], label='MATHUSLA', color='blue', s=20, alpha=0.3)
    add_envelope(plt, df_threshold_lowest_MATHUSLA['m'], df_threshold_lowest_MATHUSLA['tanb'], 'MATHUSLA', 'blue', interpolation)
    plt.xlim(min(df['m']), 10)
    # axs[0, 0].set_ylim(1e-14, 1e-6)
    # plt.title(r'A, mHC = 90GeV, cos($\alpha$-$\beta$) = 0', fontsize=20)
    plt.title(r'mH, mHC = 90GeV, cos($\alpha$-$\beta$) = 0', fontsize=20)
    plt.xlabel('mass / GeV', fontsize=15)
    plt.ylabel(r'tan$\beta$', fontsize=15)
    plt.yscale('log')
    plt.xscale('log')
    plt.legend()
    plt.tight_layout()
    plt.savefig(output_file)
    plt.show()
    plt.close()


def plot_with_envelope_tanb_all_Detector_H(csv_file, interpolation='cubic', output_file='envelope_plot.png', visible_br_lowest=0.76):
    # 删除离群点
    df = pd.read_csv(csv_file)


    # df['significance_with_4pi'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3
    # df['significance_without_4pi'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * df['visible_br_without_4pi']
    df['significance_lowest_CODEX-b'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * visible_br_lowest
    df['significance_lowest_MATHUSLA'] = df['detector_acceptance_MATHUSLA']* fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * visible_br_lowest
    df['significance_lowest_MATHUSLA'] = df['detector_acceptance_SHiP']* fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * visible_br_lowest
    # df['simple_br'] = df['detector_acceptance'] * fcal.calculate_Br(df['m'], df['tanb'], 0.104, 0.653) * 300 * df['Cross_section_fb'] * 1e3 * df['visible_br_without_4pi']
    # fig, axs = plt.subplots(2, 2, figsize=(20, 15))
    # df_threshold = df[df['significance_with_4pi'] > 3]
    # df_threshold_without_4pi = df[df['significance_without_4pi'] > 3]
    df_threshold_lowest_CODEX_b = df[df['significance_lowest_CODEX-b'] > 3]
    df_threshold_lowest_MATHUSLA = df[df['significance_lowest_MATHUSLA'] > 3]
    # df_threshold_simple_br = df[df['simple_br'] > 3]
    def add_envelope(ax, x, y, label, color, interpolation='cubic'):
        """
        Add an envelope to the given axis and interpolate the upper and lower envelopes.

        Parameters:
        - ax: matplotlib axis object
        - x: x data (m)
        - y: y data (sin^2(tanb))
        - label: label for the envelope
        - color: color of the envelope
        - interpolation: interpolation method ('cubic' or 'quadratic')
        """
        # Sort data by x
        sorted_indices = np.argsort(x)
        x = np.array(x)[sorted_indices]
        y = np.array(y)[sorted_indices]

        # Sort upper and lower envelope points
        unique_x = np.unique(x)
        upper_envelope_x = []
        upper_envelope_y = []
        lower_envelope_x = []
        lower_envelope_y = []

        for ux in unique_x:
            mask = x == ux
            upper_envelope_x.append(ux)
            upper_envelope_y.append(y[mask].max())  
            lower_envelope_x.append(ux)
            lower_envelope_y.append(y[mask].min())  

        # Interpolate the upper and lower envelope
        x_interp = np.linspace(min(upper_envelope_x), max(upper_envelope_x), 500)
        upper_interp = interp1d(upper_envelope_x, upper_envelope_y, kind=interpolation, fill_value="extrapolate")
        lower_interp = interp1d(lower_envelope_x, lower_envelope_y, kind=interpolation, fill_value="extrapolate")

        # generate interpolated values
        y_upper = upper_interp(x_interp)
        y_lower = lower_interp(x_interp)

        ax.plot(x_interp, y_upper, color=color, linestyle='--', alpha=0.8)
        ax.plot(x_interp, y_lower, color=color, linestyle='--', alpha=0.8)
        ax.fill_between(x_interp, y_lower, y_upper, color=color, alpha=0.1, label=f'Envelope Area ({label})')

    # Remove rows with NaN or infinite values in relevant columns
    # df_threshold_lowest_br = df_threshold_lowest_br.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance', 'significance_lowest_br'])
    df_threshold_lowest_CODEX_b = df_threshold_lowest_CODEX_b.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance', 'significance_lowest_CODEX-b'])
    df_threshold_lowest_MATHUSLA = df_threshold_lowest_MATHUSLA.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance_MATHUSLA', 'significance_lowest_MATHUSLA'])
    # df_threshold_without_4pi = df_threshold_without_4pi.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance', 'significance_without_4pi'])
    # Subplot 1: No 4Pi
    plt.figure(figsize=(15, 15))
    plt.scatter(df_threshold_lowest_CODEX_b['m'], df_threshold_lowest_CODEX_b['tanb'], label='CODEX-b', color='green', s=20, alpha=0.3)
    add_envelope(plt, df_threshold_lowest_CODEX_b['m'], df_threshold_lowest_CODEX_b['tanb'], 'CODEX-b', 'green', interpolation)
    plt.scatter(df_threshold_lowest_MATHUSLA['m'], df_threshold_lowest_MATHUSLA['tanb'], label='MATHUSLA', color='blue', s=20, alpha=0.3)
    add_envelope(plt, df_threshold_lowest_MATHUSLA['m'], df_threshold_lowest_MATHUSLA['tanb'], 'MATHUSLA', 'blue', interpolation)
    plt.xlim(min(df['m']), 10)
    # axs[0, 0].set_ylim(1e-14, 1e-6)
    # plt.title(r'A, mHC = 90GeV, cos($\alpha$-$\beta$) = 0', fontsize=20)
    plt.title(r'H, mHC = 600GeV, cos($\alpha$-$\beta$) = 1/tan$\beta$', fontsize=20)
    plt.xlabel('mass / GeV', fontsize=15)
    plt.ylabel(r'tan$\beta$', fontsize=15)
    plt.yscale('log')
    plt.xscale('log')
    plt.legend()
    plt.tight_layout()
    plt.savefig(output_file)
    plt.show()
    plt.close()
    
def plot_with_envelope_tanb_all_Detector_A(csv_file, interpolation='cubic', output_file='envelope_plot.png', visible_br_lowest=0.76):
    # 删除离群点
    df = pd.read_csv(csv_file)


    # df['significance_with_4pi'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3
    # df['significance_without_4pi'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * df['visible_br_without_4pi']
    df['significance_lowest_CODEX-b'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * visible_br_lowest
    df['significance_lowest_MATHUSLA'] = df['detector_acceptance_MATHUSLA']* fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * visible_br_lowest
    df['significance_lowest_SHiP'] = df['detector_acceptance_SHiP']* fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * visible_br_lowest
    # df['simple_br'] = df['detector_acceptance'] * fcal.calculate_Br(df['m'], df['tanb'], 0.104, 0.653) * 300 * df['Cross_section_fb'] * 1e3 * df['visible_br_without_4pi']
    # fig, axs = plt.subplots(2, 2, figsize=(20, 15))
    # df_threshold = df[df['significance_with_4pi'] > 3]
    # df_threshold_without_4pi = df[df['significance_without_4pi'] > 3]
    df_threshold_lowest_CODEX_b = df[df['significance_lowest_CODEX-b'] > 3]
    df_threshold_lowest_MATHUSLA = df[df['significance_lowest_MATHUSLA'] > 3]
    df_threshold_lowest_SHiP = df[df['significance_lowest_SHiP'] > 3]
    # df_threshold_simple_br = df[df['simple_br'] > 3]
    def add_envelope(ax, x, y, label, color, interpolation='cubic'):
        """
        Add an envelope to the given axis and interpolate the upper and lower envelopes.

        Parameters:
        - ax: matplotlib axis object
        - x: x data (m)
        - y: y data (sin^2(tanb))
        - label: label for the envelope
        - color: color of the envelope
        - interpolation: interpolation method ('cubic' or 'quadratic')
        """
        # Sort data by x
        sorted_indices = np.argsort(x)
        x = np.array(x)[sorted_indices]
        y = np.array(y)[sorted_indices]

        # Sort upper and lower envelope points
        unique_x = np.unique(x)
        upper_envelope_x = []
        upper_envelope_y = []
        lower_envelope_x = []
        lower_envelope_y = []

        for ux in unique_x:
            mask = x == ux
            upper_envelope_x.append(ux)
            upper_envelope_y.append(y[mask].max())  
            lower_envelope_x.append(ux)
            lower_envelope_y.append(y[mask].min())  

        # Interpolate the upper and lower envelope
        x_interp = np.linspace(min(upper_envelope_x), max(upper_envelope_x), 500)
        upper_interp = interp1d(upper_envelope_x, upper_envelope_y, kind=interpolation, fill_value="extrapolate")
        lower_interp = interp1d(lower_envelope_x, lower_envelope_y, kind=interpolation, fill_value="extrapolate")

        # generate interpolated values
        y_upper = upper_interp(x_interp)
        y_lower = lower_interp(x_interp)

        ax.plot(x_interp, y_upper, color=color, linestyle='--', alpha=0.8)
        ax.plot(x_interp, y_lower, color=color, linestyle='--', alpha=0.8)
        ax.fill_between(x_interp, y_lower, y_upper, color=color, alpha=0.1, label=f'Envelope Area ({label})')

    # Remove rows with NaN or infinite values in relevant columns
    # df_threshold_lowest_br = df_threshold_lowest_br.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance', 'significance_lowest_br'])
    df_threshold_lowest_CODEX_b = df_threshold_lowest_CODEX_b.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance', 'significance_lowest_CODEX-b'])
    df_threshold_lowest_MATHUSLA = df_threshold_lowest_MATHUSLA.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance_MATHUSLA', 'significance_lowest_MATHUSLA'])
    df_threshold_lowest_SHiP = df_threshold_lowest_SHiP.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance_SHiP', 'significance_lowest_SHiP']) 
    # df_threshold_without_4pi = df_threshold_without_4pi.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'tanb', 'detector_acceptance', 'significance_without_4pi'])
    # Subplot 1: No 4Pi
    plt.figure(figsize=(15, 15))
    plt.scatter(df_threshold_lowest_CODEX_b['m'], df_threshold_lowest_CODEX_b['tanb'], label='CODEX-b', color='green', s=20, alpha=0.3)
    add_envelope(plt, df_threshold_lowest_CODEX_b['m'], df_threshold_lowest_CODEX_b['tanb'], 'CODEX-b', 'green', interpolation)
    plt.scatter(df_threshold_lowest_MATHUSLA['m'], df_threshold_lowest_MATHUSLA['tanb'], label='MATHUSLA', color='blue', s=20, alpha=0.3)
    add_envelope(plt, df_threshold_lowest_MATHUSLA['m'], df_threshold_lowest_MATHUSLA['tanb'], 'MATHUSLA', 'blue', interpolation)
    plt.scatter(df_threshold_lowest_SHiP['m'], df_threshold_lowest_SHiP['tanb'], label='SHiP', color='red', s=20, alpha=0.3)
    add_envelope(plt, df_threshold_lowest_SHiP['m'], df_threshold_lowest_SHiP['tanb'], 'SHiP', 'red', interpolation)
    print(df['m'][:5])
    plt.xlim(min(df['m']), 10)
    # axs[0, 0].set_ylim(1e-14, 1e-6)
    # plt.title(r'A, mHC = 90GeV, cos($\alpha$-$\beta$) = 0', fontsize=20)
    plt.title(r'mH, mHC = 90GeV, cos($\alpha$-$\beta$) = 0', fontsize=20)
    plt.xlabel('mass / GeV', fontsize=15)
    plt.ylabel(r'tan$\beta$', fontsize=15)
    plt.yscale('log')
    plt.xscale('log')
    plt.legend()
    plt.tight_layout()
    plt.savefig(output_file)
    plt.show()
    plt.close()

In [16]:
# csv_file = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_CODEX_Original/2025-06-01_2HDM-H/LLP_data/2025-06-03_H.csv'
# df_test = pd.read_csv(csv_file)
csv_1 = '/home/srp/Documents/LLP_DATA/2025-08-08_2HDM-A/2025-08-12_detected_combined_precise_file.csv'
# csv_2 = '/Volumes/USB_DRIVE/NEW/2025-06-03_A.csv'
# csv_all = '2HDM_A_test.csv'
# df_all = pd.concat([pd.read_csv(csv_1), pd.read_csv(csv_2)])
# df_all.to_csv('2HDM_A_test.csv')
# print(max(df_test['m']))
# df_2 = pd.read_csv(csv_2)
# print(df_2['Cross_section_fb']/df_2['Cross_section'])
plt.figure(figsize=(15, 15))
plt.scatter(pd.read_csv(csv_1)['m'], np.pow(10, pd.read_csv(csv_1)['tanb']), label='2HDM-A-SHiP', color='green', s=20, alpha=0.3)
print(np.min(np.pow(10, pd.read_csv(csv_1)['tanb'])))
# plt.xlim(0, 1)
plt.yscale('log')
plt.xscale('log')
plt.xlabel('mass / GeV', fontsize=15)
plt.ylabel(r'tan$\beta$', fontsize=15)
plt.title(r'2HDM-A-SHiP, mHC = 90GeV, cos($\alpha$-$\beta$) = 0', fontsize=20)
plt.legend()
plt.tight_layout()
plt.savefig('2HDM_A_SHiP.png')
# plt.show()
plt.close()
# plot_with_envelope_tanb_all_Detector_A(csv_1, 'nearest-up', '2HDM_A_SHiP.png')
# plot_with_envelope_tanb_all_Detector_A(csv_2, 'nearest-up', '2HDM_A_2.png')
# plot_with_envelope_tanb_all_Detector_A(csv_all, 'nearest-up', '2HDM_A_all.png')
# fplot.plot_with_envelope_tanb(csv_file, 'cubic', 'HDM1_Test.png')

10000.0


In [40]:
from scipy.signal import find_peaks
def fine_envelope(x, y):
    x = np.array(x)
    y = np.array(y)


    peaks, _ = find_peaks(y)  # 上包络线
    troughs, _ = find_peaks(-y)  # 下包络线
    return x[peaks], y[peaks], x[troughs], y[troughs]

In [63]:
def plot_with_envelope_DDC_PRA(csv_file, DDC_file, interpolation='cubic', output_file='envelope_plot.png'):
    # 删除离群点
    df = pd.read_csv(csv_file)
    df_ddc = pd.read_csv(DDC_file)

    df['significance_with_4pi'] = df['detector_acceptance'] * fcal.calculate_Br(df['m'], np.sin(df['theta_input'])**2, 0.104, 0.653) * 300 * df['Cross_section_fb'] * 1e3 * df['visible_br']
    fig, axs = plt.subplots(1, 2, figsize=(20, 15))
    df_threshold = df[df['significance_with_4pi'] > 3]
    # print(df_threshold[:3])
    print(df_ddc[:3])
    def add_envelope(ax, x, y, label, color, interpolation='cubic'):
        """
        Add an envelope to the given axis and interpolate the upper and lower envelopes.

        Parameters:
        - ax: matplotlib axis object
        - x: x data (m)
        - y: y data (sin^2(theta_input))
        - label: label for the envelope
        - color: color of the envelope
        - interpolation: interpolation method ('cubic' or 'quadratic')
        """
        # Sort data by x
        sorted_indices = np.argsort(x)
        x = np.array(x)[sorted_indices]
        y = np.array(y)[sorted_indices]

        # Sort upper and lower envelope points
        unique_x = np.unique(x)
        upper_envelope_x = []
        upper_envelope_y = []
        lower_envelope_x = []
        lower_envelope_y = []

        for ux in unique_x:
            mask = x == ux
            upper_envelope_x.append(ux)
            upper_envelope_y.append(y[mask].max())  
            lower_envelope_x.append(ux)
            lower_envelope_y.append(y[mask].min())  

        # Interpolate the upper and lower envelope
        x_interp = np.linspace(min(upper_envelope_x), max(upper_envelope_x), 500)
        upper_interp = interp1d(upper_envelope_x, upper_envelope_y, kind=interpolation, fill_value="extrapolate")
        lower_interp = interp1d(lower_envelope_x, lower_envelope_y, kind=interpolation, fill_value="extrapolate")

        # generate interpolated values
        y_upper = upper_interp(x_interp)
        y_lower = lower_interp(x_interp)

        ax.plot(x_interp, y_upper, color=color, linestyle='--', alpha=0.8)
        ax.plot(x_interp, y_lower, color=color, linestyle='--', alpha=0.8)
        ax.fill_between(x_interp, y_lower, y_upper, color=color, alpha=0.1, label=f'Envelope Area ({label})')

    # Remove rows with NaN or infinite values in relevant columns
    # df_threshold_lowest_br = df_threshold_lowest_br.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'theta_input', 'detector_acceptance', 'significance_lowest_br'])
    df_threshold = df_threshold.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'theta_input', 'detector_acceptance', 'significance_with_4pi'])
    # df_threshold_without_4pi = df_threshold_without_4pi.replace([np.inf, -np.inf], np.nan).dropna(subset=['m', 'theta_input', 'detector_acceptance', 'significance_without_4pi'])
    # Subplot 1: No 4Pi
    add_envelope(axs[0], df_threshold['m'], np.sin(df_threshold['theta_input'])**2, 'PRA Result', 'red', interpolation)
    axs[0].set_xlim(0.1, 5)
    axs[0].set_ylim(1e-12, 1e-6)
    axs[0].set_title(r'PRA Result', fontsize=20)
    axs[0].set_xlabel('mass / GeV', fontsize=15)
    axs[0].set_ylabel(r'sin$\theta^2$', fontsize=15)
    axs[0].set_yscale('log')
    # axs[0, 0].set_xscale('log')
    axs[0].legend()

    # Subplot 2: With 4Pi
    # add_envelope(axs[1], df_ddc['m'], np.sin(df_ddc['theta_input'])**2, 'DDC Result', 'blue', interpolation)
    df_ddc = df_ddc.sort_values(by='m')
    ddc_m_peak, ddc_theta_peak, ddc_m_trough, ddc_theta_trough = fine_envelope(df_ddc['m'], df_ddc['theta_input'])
    upper = np.interp(df_ddc['m'], ddc_m_peak, np.sin(ddc_theta_peak)**2)
    lower = np.interp(df_ddc['m'], ddc_m_trough, np.sin(ddc_theta_trough)**2)
    axs[1].plot(ddc_m_peak, np.sin(ddc_theta_peak)**2, 'o', color='blue', linestyle="--", alpha=0.5)
    axs[1].plot(ddc_m_trough, np.sin(ddc_theta_trough)**2, 'o', color='blue', linestyle="--", alpha=0.5)
    axs[1].fill_between(df_ddc['m'], lower, upper, color="blue", alpha=0.2, label="DDC Result")
    # axs[1].scatter(df_ddc['m'], np.sin(df_ddc['theta_input'])**2, color='blue', label='DDC Result', alpha=0.5)
    print('fine')
    axs[1].plot()
    
    add_envelope(axs[1], df_threshold['m'], np.sin(df_threshold['theta_input'])**2, 'PRA Result', 'red', interpolation)
    axs[1].set_xlim(0.1, 5)
    axs[1].set_ylim(1e-12, 1e-6)
    axs[1].set_title(r'Compare of PRA and DDC', fontsize=20)
    axs[1].set_xlabel('mass / GeV', fontsize=15)
    axs[1].set_ylabel(r'sin$\theta^2$', fontsize=15)
    axs[1].set_yscale('log')
    # axs[0, 1].set_xscale('log')
    axs[1].legend()


    plt.tight_layout()
    plt.savefig(output_file)
    plt.show()
    plt.close()

In [64]:
DDC_file = "/Users/shiyuzhe/Documents/University/LLP/Programs/Git_upload/LLP_DATA/DDC_Higgs_Portal.csv"
PRA_file = '/Users/shiyuzhe/Documents/University/LLP/Programs/Git_upload/LLP_DATA/combined_4_10-15.csv'
plot_with_envelope_DDC_PRA(PRA_file, DDC_file, 'nearest-up', 'DDC_PRA_Higgs_Portal.png')

          m  theta_input
0  0.104167     0.000374
1  0.115657     0.000352
2  0.130144     0.000335
fine


/var/folders/kt/7k22p1ns6l7ggtc7ck9nkqbw0000gn/T/ipykernel_26355/2354302072.py:96: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [11]:
# print(fcal.calcu_Br_B_to_H(5, 1e2, 1e-2, 600))
tanb = 5
x = np.logspace(-1, np.log(4.18), 100)
y = fcal.calcu_Br_B_to_H(x, tanb, 1/tanb)
# plt.close()
plt.figure(figsize=(8, 6))
plt.plot(x, y)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('m_H [GeV]')
plt.ylabel('Br(B -> H)')
plt.title(f'Br(B -> H) vs m_H_tanb_{tanb}')
# plt.grid()
plt.show()
plt.savefig('Br_B_to_H_vs_mH_tanb.png', dpi=300, bbox_inches='tight')
print(y[:5])
# print(fcal.calcu_Br_B_H_2HDM_I_simple(10, 1e2))

/var/folders/kt/7k22p1ns6l7ggtc7ck9nkqbw0000gn/T/ipykernel_1279/340126615.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


[1.62378685 1.62356427 1.62331506 1.62303605 1.62272368]


In [13]:
fcal.calcu_Br_B_to_H(0.1, 5, 0.2, 600)

1.623786853584878

In [5]:
# /Users/shiyuzhe/Documents/University/LLP/Second_Term/pythia8/BtoKa/auto_data/Ctau_Br/2024-09-17-csv/original_data

In [ ]:
filename = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/2025-4-10-4-14/combined.csv'
df = pd.read_csv(filename)
df['BR_CAL'] = fcal.calculate_Br(df['m'], np.sin(df['theta_input'])**2, 0.104, 0.653)
df_m_eq_1 = df[df['m'] == 1]
df_m_eq_2 = df[df['m'] == 2]
plt.figure(figsize=(10, 6))
plt.scatter(df_m_eq_1['tau_input']/(3e5), df_m_eq_1['BR_CAL'], color='red', alpha=0.7, label='BR_CAL for m=1')
plt.scatter(df_m_eq_2['tau_input']/(3e5), df_m_eq_2['BR_CAL'], color='blue', alpha=0.7, label='BR_CAL for m=2')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('tau0/mm c-1', fontsize=14)
plt.ylabel('BR_CAL', fontsize=14)
plt.title('BR_CAL for m', fontsize=16)
plt.legend()
plt.grid(True, which="both", linestyle='--', linewidth=0.5)
plt.show()
plt.savefig("BR_M.png")
plt.close()


In [ ]:
DDC_file = "/Users/shiyuzhe/Documents/University/LLP/Programs/Git_upload/LLP_DATA/DDC_Higgs_Portal.csv"
# 读取数据
filename = '/Users/shiyuzhe/Documents/University/LLP/Programs/Git_upload/LLP_DATA/combined_4_10-15.csv'
df = pd.read_csv(filename)
df_ddc = pd.read_csv(DDC_file)
# 计算 significance
df['significance_with_4pi'] = df['detector_acceptance'] * fcal.calculate_Br(df['m'], np.square(np.sin(df['theta_input']))) * 300 * df['Cross_section_fb'] * 1e3 * df['visible_br']

size_scatter = 30
fig, axs = plt.subplots(1, 2, figsize=(15, 10))
df_threshold = df[df['significance_with_4pi'] > 3]
df_lowest = df[df['significance_lowest'] > 3]
# df_lowest_MATHUSLA = df[df['significance_lowest_MATHUSLA'] > 3]


axs[0].scatter(df['m'], np.square(np.sin(df['theta_input'])), label='All Data', color='green', s=size_scatter, alpha=0.2)
axs[0].set_xlim(0.1, 3)
axs[0].set_ylim(1e-14, 1e-6)
axs[0].set_title(r'ALL DATA', fontsize=20)
axs[0].set_xlabel('mass / GeV', fontsize=15)
axs[0].set_ylabel(r'$sin^2 \theta$', fontsize=15)
axs[0].set_yscale('log')
# axs[0].set_xscale('log')
axs[0].legend()

# 子图 2：有阈值的散点图
axs[1].scatter(df['m'], np.square(np.sin(df['theta_input'])), label='All Data', color='green', s=size_scatter, alpha=0.1)
axs[1].scatter(df_lowest['m'], np.square(np.sin(df_lowest['theta_input'])), label='0.95 Significance', color='blue', s=size_scatter, alpha=0.2)
# axs[1].scatter(df_threshold['m'], df_threshold['tanb'], label=r'With 4Pi ($3\sigma$)', color='blue', s=size_scatter, alpha=0.5)
axs[1].set_xlim(0.1, 3)
axs[1].set_ylim(1e-14, 1e-6)
axs[1].set_title(r'Data Process', fontsize=20)
axs[1].set_xlabel('mass / GeV', fontsize=15)
axs[1].set_ylabel(r'$sin^2 \theta$', fontsize=15)
axs[1].set_yscale('log')
# axs[1].set_xscale('log')
axs[1].legend()

# 调整布局并显示图像
plt.tight_layout()
plt.show()

# 子图 4：两种数据叠加，有阈值的透明度较低
# axs[1, 1].scatter(df_lowest['m'], np.sin(df_lowest['tanb'])**2, label='With Threshold(lowest visible Br)', color='red', s=size_scatter, alpha=0.2)
# axs[1, 1].scatter(df_threshold['m'], np.sin(df_threshold['tanb'])**2, label=r'With Threshold ($3\sigma$)', color='blue', s=size_scatter, alpha=0.5)

# axs[1, 1].set_xlim(0, 5)
# axs[1, 1].set_ylim(1e-14, 1e-6)
# axs[1, 1].set_title('Both Data (Low Alpha Lowest) test', fontsize=20)
# axs[1, 1].set_xlabel('mass / GeV', fontsize=15)
# axs[1, 1].set_ylabel('theta^2', fontsize=15)
# axs[1, 1].set_yscale('log')
# axs[1, 1].legend()
plt.savefig('B_Test.jpg')
plt.show()
plt.close()

In [8]:

# 读取数据
filename = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_CODEX_Original/2025-05-20_Higgs_Portal/2025-05-20_detected_combined_precise_file.csv'
df = pd.read_csv(filename)

# 计算 significance
df['significance300'] = df['detector_acceptance'] * fcal.calculate_Br(df['m'], np.square(np.sin(df['theta_input']))) * 300 * df['Cross_section_fb'] * 1e3
df['significance10'] = df['detector_acceptance'] * fcal.calculate_Br(df['m'], np.square(np.sin(df['theta_input']))) * 10 * df['Cross_section_fb'] * 1e3
# df['significance_lowest'] = df['detector_acceptance'] * 0.102 * 300 * df['Cross_section_fb'] * 1e3 * 0.76
# print(fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600)[:10])
# df['significance_lowest'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * 0.76
# df['significance_lowest_MATHUSLA'] = df['detector_acceptance_MATHUSLA'] * fcal.calcu_Br_B_to_H(df['m'], np.sin(df['tanb'])**2, 0.104, 0.653) * 300 * df['Cross_section_fb'] * 1e3 * 0.76

# df['significance_without_4pi'] = df['detector_acceptance'] * fcal.calculate_Br(df['m'], np.sin(df['tanb'])**2, 0.104, 0.653) * 300 * df['Cross_section_fb'] * 1e3 * df['visible_br_without_4pi']
# The 0.102 is the Br of B->X_c e v(See CODEX-b paper or Light Scalar at FASER for details)
# The 0.653 is g, which is a constant decribing the coupling of weak interaction
# The 1e3 is because I wrote pythia8 wrong, the \sigma need to multiply 1e3
# The 383271000.0 is a average cross section of B meson production in pp collision at 14TeV, Will be replaced by thr real cross section of each pariticle in the future
# print(df['significance_with_4pi'][1000:1005])
# 创建子图
size_scatter = 30
fig, axs = plt.subplots(1, 2, figsize=(15, 10))
df_300 = df[df['significance300'] > 3]
df_10 = df[df['significance10'] > 3]
# df_lowest_MATHUSLA = df[df['significance_lowest_MATHUSLA'] > 3]


axs[0].scatter(df_300['m'], np.square(np.sin(df_300['theta_input'])), label='All Data', color='green', s=size_scatter, alpha=0.2)
axs[0].set_xlim(0.1, 3)
axs[0].set_ylim(1e-14, 1e-6)
axs[0].set_title(r'300fb-1', fontsize=20)
axs[0].set_xlabel('mass / GeV', fontsize=15)
axs[0].set_ylabel(r'$sin^2 \theta$', fontsize=15)
axs[0].set_yscale('log')
# axs[0].set_xscale('log')
axs[0].legend()

# 子图 2：有阈值的散点图
axs[1].scatter(df_10['m'], np.square(np.sin(df_10['theta_input'])), label='All Data', color='green', s=size_scatter, alpha=0.1)
# axs[1].scatter(df_lowest['m'], np.square(np.sin(df_lowest['theta_input'])), label='0.95 Significance', color='blue', s=size_scatter, alpha=0.2)
# axs[1].scatter(df_threshold['m'], df_threshold['tanb'], label=r'With 4Pi ($3\sigma$)', color='blue', s=size_scatter, alpha=0.5)
axs[1].set_xlim(0.1, 3)
axs[1].set_ylim(1e-14, 1e-6)
axs[1].set_title(r'10fb-1', fontsize=20)
axs[1].set_xlabel('mass / GeV', fontsize=15)
axs[1].set_ylabel(r'$sin^2 \theta$', fontsize=15)
axs[1].set_yscale('log')
# axs[1].set_xscale('log')
axs[1].legend()

# 调整布局并显示图像
plt.tight_layout()
plt.show()

# 子图 4：两种数据叠加，有阈值的透明度较低
# axs[1, 1].scatter(df_lowest['m'], np.sin(df_lowest['tanb'])**2, label='With Threshold(lowest visible Br)', color='red', s=size_scatter, alpha=0.2)
# axs[1, 1].scatter(df_threshold['m'], np.sin(df_threshold['tanb'])**2, label=r'With Threshold ($3\sigma$)', color='blue', s=size_scatter, alpha=0.5)

# axs[1, 1].set_xlim(0, 5)
# axs[1, 1].set_ylim(1e-14, 1e-6)
# axs[1, 1].set_title('Both Data (Low Alpha Lowest) test', fontsize=20)
# axs[1, 1].set_xlabel('mass / GeV', fontsize=15)
# axs[1, 1].set_ylabel('theta^2', fontsize=15)
# axs[1, 1].set_yscale('log')
# axs[1, 1].legend()
plt.savefig('DFHiggs_Test.jpg')
plt.show()
plt.close()

/tmp/ipykernel_5846/3146647678.py:52: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_5846/3146647678.py:66: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [8]:

# 读取数据
filename = '/Users/shiyuzhe/Documents/University/LLP/Programs/Git_upload/LLP_DATA/2HDM1_B.csv'
df = pd.read_csv(filename)

# 计算 significance
df['significance_with_4pi'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * df['visible_br']
df['significance_lowest'] = df['detector_acceptance'] * fcal.calculate_Br(df['m'], df['tanb'], 0.104) * 300 * df['Cross_section_fb'] * 1e3 * 0.76
# df['significance_lowest'] = df['detector_acceptance'] * 0.102 * 300 * df['Cross_section_fb'] * 1e3 * 0.76
# print(fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600)[:10])
# df['significance_lowest'] = df['detector_acceptance'] * fcal.calcu_Br_B_to_H(df['m'], df['tanb'], 1/df['tanb'], 600) * 300 * df['Cross_section_fb'] * 1e3 * 0.76
# df['significance_lowest_MATHUSLA'] = df['detector_acceptance_MATHUSLA'] * fcal.calcu_Br_B_to_H(df['m'], np.sin(df['tanb'])**2, 0.104, 0.653) * 300 * df['Cross_section_fb'] * 1e3 * 0.76

# df['significance_without_4pi'] = df['detector_acceptance'] * fcal.calculate_Br(df['m'], np.sin(df['tanb'])**2, 0.104, 0.653) * 300 * df['Cross_section_fb'] * 1e3 * df['visible_br_without_4pi']
# The 0.102 is the Br of B->X_c e v(See CODEX-b paper or Light Scalar at FASER for details)
# The 0.653 is g, which is a constant decribing the coupling of weak interaction
# The 1e3 is because I wrote pythia8 wrong, the \sigma need to multiply 1e3
# The 383271000.0 is a average cross section of B meson production in pp collision at 14TeV, Will be replaced by thr real cross section of each pariticle in the future
# print(df['significance_with_4pi'][1000:1005])
# 创建子图
size_scatter = 30
fig, axs = plt.subplots(1, 2, figsize=(20, 15))
df_threshold = df[df['significance_with_4pi'] > 3]
df_lowest = df[df['significance_lowest'] > 3]
# df_lowest_MATHUSLA = df[df['significance_lowest_MATHUSLA'] > 3]


axs[0].scatter(df_threshold['m'], df_threshold['tanb'], label='More Precise Br', color='green', s=size_scatter, alpha=0.2)
axs[0].set_xlim(0.1, 4)
# axs[0].set_ylim(1e-14, 1e-6)
axs[0].set_title(r'Precise Br', fontsize=20)
axs[0].set_xlabel('mass / GeV', fontsize=15)
axs[0].set_ylabel('tanb', fontsize=15)
axs[0].set_yscale('log')
axs[0].set_xscale('log')
axs[0].legend()

# 子图 2：有阈值的散点图
axs[1].scatter(df_lowest['m'], df_lowest['tanb'], label='Simple Br', color='blue', s=size_scatter, alpha=0.2)
# axs[1].scatter(df_threshold['m'], df_threshold['tanb'], label=r'With 4Pi ($3\sigma$)', color='blue', s=size_scatter, alpha=0.5)
axs[1].set_xlim(0.1, 4)
# axs[1].set_ylim(1e-14, 1e-6)
axs[1].set_title(r'Simple Br', fontsize=20)
axs[1].set_xlabel('mass / GeV', fontsize=15)
axs[1].set_ylabel('tanb', fontsize=15)
axs[1].set_yscale('log')
axs[1].set_xscale('log')
axs[1].legend()

# 调整布局并显示图像
plt.tight_layout()
plt.show()

# 子图 4：两种数据叠加，有阈值的透明度较低
# axs[1, 1].scatter(df_lowest['m'], np.sin(df_lowest['tanb'])**2, label='With Threshold(lowest visible Br)', color='red', s=size_scatter, alpha=0.2)
# axs[1, 1].scatter(df_threshold['m'], np.sin(df_threshold['tanb'])**2, label=r'With Threshold ($3\sigma$)', color='blue', s=size_scatter, alpha=0.5)

# axs[1, 1].set_xlim(0, 5)
# axs[1, 1].set_ylim(1e-14, 1e-6)
# axs[1, 1].set_title('Both Data (Low Alpha Lowest) test', fontsize=20)
# axs[1, 1].set_xlabel('mass / GeV', fontsize=15)
# axs[1, 1].set_ylabel('theta^2', fontsize=15)
# axs[1, 1].set_yscale('log')
# axs[1, 1].legend()
plt.savefig('HDM1_Test_log_2Br.jpg')
plt.show()
plt.close()

/var/folders/kt/7k22p1ns6l7ggtc7ck9nkqbw0000gn/T/ipykernel_90490/1436378879.py:51: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/kt/7k22p1ns6l7ggtc7ck9nkqbw0000gn/T/ipykernel_90490/1436378879.py:65: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [27]:
csv_file = '/home/srp/Documents/program/stonerainphilo-Light_scalar_decay/Combined_Code/2HDM-I_A_21.csv'
df = pd.read_csv(csv_file)
plt.figure(figsize=(15, 15))
plt.scatter(df['tanb'], df['ltime'], label='A')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('tanb / GeV', fontsize=15)
plt.ylabel('lifetime / mm', fontsize=15)
plt.title('2HDM-I A Lifetime', fontsize=20)
plt.legend()
plt.tight_layout()
plt.savefig('2HDM_I_A_Lifetime.png')
plt.show()
plt.close()

/tmp/ipykernel_38975/3096891314.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:


csv_file = '/home/srp/Documents/program/position_read_analyse_1.3/2HDM-I_A_8.csv_Sorted.csv'
df = pd.read_csv(csv_file)
plt.figure(figsize=(15, 15))
plt.scatter(df['mA'], df['tanb'], label='2HDM-A SHiP', color='green', s=20, alpha=0.3)
plt.xlim(0.1, 5)
plt.yscale('log')
plt.xscale('log')
plt.xlabel('mass / GeV', fontsize=15)
plt.ylabel(r'tan$\beta$', fontsize=15)
plt.title(r'Higgs Scalar', fontsize=20)
plt.legend()
plt.tight_layout()
plt.savefig('SHiP_2HDM-A_test_raw-8.png')
plt.show()
plt.close()

/tmp/ipykernel_38975/3998744201.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [19]:
csv_file = '/home/srp/Documents/LLP_DATA/2025-08-21_2HDM-A/2025-08-23_detected_combined_precise_file.csv'
df = pd.read_csv(csv_file)
plt.figure(figsize=(15, 15))
plt.scatter(df['m'], df['tanb_input'], label='2HDM-A SHiP', color='green', s=20, alpha=0.3)
plt.xlim(0.1, 5)
plt.yscale('log')
plt.xscale('log')
plt.xlabel('mass / GeV', fontsize=15)
plt.ylabel(r'tan$\beta$', fontsize=15)
plt.title(r'Higgs Scalar', fontsize=20)
plt.legend()
plt.tight_layout()
plt.savefig('SHiP_2HDM-A_test.png')
plt.show()
plt.close()

/tmp/ipykernel_38975/517019487.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:

csv1 = 'FINAL_DATA/csv/MATHUSLA_A.csv'
csv2 = 'FINAL_DATA/csv/CODEXb_A.csv'
# csv3 = '/home/srp/Documents/program/position_read_analyse_1.3/FINAL_DATA/csv/SHiP_A.csv'
csv4 = 'FINAL_DATA/csv/SHiP_A_upp.csv'
df1 = pd.read_csv(csv1)
df2 = pd.read_csv(csv2)
# df3 = pd.read_csv(csv3)
df4 = pd.read_csv(csv4)
# Create a figure and plot both datasets
plt.figure(figsize=(20, 16))
plt.plot(df1['m'], df1['tanb'], color='red', label='MATHUSLA', alpha=0.7, linestyle='-')
plt.plot(df2['m'], df2['tanb'], color='blue', label='CODEXb', alpha=0.7, linestyle='-')
# plt.plot(df3['m'], df3['tanb'], color='green', label='SHiP', alpha=0.7, linestyle='-')
plt.plot(df4['m'], df4['tanb'], color='green', label='SHiP Upper Bound(uncertain)', alpha=0.7, linestyle='--')
plt.xlabel('mA(GEV)')
plt.ylabel('tanb')
plt.title(r'mH, mHC = 90GeV, cos($\alpha$-$\beta$) = 0', fontsize=16)
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which='both', linestyle='--', linewidth=2)
plt.legend()
plt.show()
plt.savefig('FINAL_DATA/Figs/S_M_C_Comparison_A.png', dpi=300, bbox_inches='tight')
plt.close()

/var/folders/kt/7k22p1ns6l7ggtc7ck9nkqbw0000gn/T/ipykernel_6346/4277542162.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [9]:
# csv1 = '/home/srp/Documents/program/position_read_analyse_1.3/FINAL_DATA/csv/MATHUSLA_A.csv'
csv2 = '/Users/shiyuzhe/Documents/University/LLP/Second_Term/pythia8/BtoKa/Code/position_read_analyse_1.3/FINAL_DATA/csv/CODEXb_Higgs_Scalar.csv'
csv3 = '/Users/shiyuzhe/Documents/University/LLP/Second_Term/pythia8/BtoKa/Code/position_read_analyse_1.3/FINAL_DATA/csv/SHiP_Higgs_Scalar.csv'
# df1 = pd.read_csv(csv1)
df2 = pd.read_csv(csv2)
df3 = pd.read_csv(csv3)
# Create a figure and plot both datasets
plt.figure(figsize=(8, 6))
# plt.plot(df1['m'], df1['tanb'], color='red', label='MATHUSLA', alpha=0.7)
plt.plot(df2['m'], df2['sintheta2'], color='blue', label='CODEXb', alpha=0.7)
plt.plot(df3['m'], df3['sintheta2'], color='green', label='SHiP', alpha=0.7)
plt.xlabel('m(GEV)')
plt.ylabel(r'sin$\theta^2$')
plt.title(r'Higgs Scalar', fontsize=16)
plt.xlim(0.1, 5)
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()
plt.show()
plt.savefig('FINAL_DATA/Figs/S_C_Comparison_Higgs_Scalar.png', dpi=300, bbox_inches='tight')
plt.close()

/var/folders/kt/7k22p1ns6l7ggtc7ck9nkqbw0000gn/T/ipykernel_6346/4263661492.py:20: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [1]:
csv_file = '/home/srp/Documents/LLP_DATA/2025-09-08_2HDM-S-2/2025-09-09_detected_combined_precise_file.csv'
df = pd.read_csv(csv_file)
df['significance_lowest'] = df['detector_acceptance_SHiP'] * fcal.calculate_Br(df['m'], df['theta'], 0.104) * df['Cross_section_fb'] * df['cross_section']  * 0.76

# size_scatter = 30
fig, axs = plt.subplots(1, 2, figsize=(20, 15))
# df_threshold = df[df['significance_with_4pi'] > 3]
df_lowest = df[df['significance_lowest'] > 3]
plt.figure(figsize=(15, 15))
plt.scatter(df_lowest['m'], np.sin(df_lowest['theta'])**2, label='2HDM-S SHiP', color='green', s=20, alpha=0.3)
plt.xlim(0.1, 5)
plt.yscale('log')
plt.xscale('log')
plt.xlabel('mass / GeV', fontsize=15)
plt.ylabel(r'sin\theta ^2$', fontsize=15)
plt.title(r'Higgs Scalar', fontsize=20)
plt.legend()
plt.tight_layout()
plt.savefig('SHiP_2HDM-S_test.png')
plt.show()
plt.close()

NameError: name 'pd' is not defined

In [20]:
def merge_csv_files(directory, output_file):
    # csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    
    data_frames = []
    
    for file in directory:
        file_path = os.path.join(file)
        df = pd.read_csv(file_path)
        data_frames.append(df)
    

    merged_df = pd.concat(data_frames, ignore_index=True)
    
    merged_df.to_csv(output_file, index=False)


